In [6]:
import torch, struct, os
import numpy as np
import tensorflow as tf

2024-01-30 00:59:31.242512: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-30 00:59:31.262410: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 00:59:31.262433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 00:59:31.262946: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 00:59:31.266139: I tensorflow/core/platform/cpu_feature_guar

In [2]:
state_dict = torch.load("../models/dnn_mnist_pt.pth", map_location=torch.device("cpu"))
list_vars = state_dict

In [5]:
fname_out = "../models/ggml-model-f32-big-endian.bin"
pack_fmt = "!i"


fout = open(fname_out, "wb")
fout.write(struct.pack(pack_fmt, 0x67676d6c)) # magic: ggml in hex

for name in list_vars.keys():
    data = list_vars[name].squeeze().numpy()
    print("Processing variable: " + name + " with shape: ", data.shape) 
    n_dims = len(data.shape)
   
    fout.write(struct.pack(pack_fmt, n_dims))
    
    data = data.astype(np.float32)
    for i in range(n_dims):
        fout.write(struct.pack(pack_fmt, data.shape[n_dims - 1 - i]))

    # data
    data = data.astype(">f4")
    data.tofile(fout)

fout.close()

Processing variable: fc1.weight with shape:  (56, 784)
Processing variable: fc1.bias with shape:  (56,)
Processing variable: fc2.weight with shape:  (10, 56)
Processing variable: fc2.bias with shape:  (10,)


In [7]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [25]:
data_file = "input"
def load_img_from_file(data_file="input", show=False):
    try:
        with open(data_file, 'rb') as file:
            buf = file.read()
    except Exception as e:
        print(e)
        return None, e

    digits = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    
    if show:
        c = ""
        for row in range(28):
            for col in range(28):
                if buf[row * 28 + col] > 230:
                    c += "&"
                else:
                    c += "-"
            c += "\n"
        print(c)

    return digits, None

def save_img_to_file(image, data_file = "input"):
    try:
        # Convert to bytes
        image_bytes = image.astype('uint8').tobytes()
        
        # Write to file
        with open(data_file, 'wb') as file:
            file.write(image_bytes)
        
        print(f"Image saved to {data_file}")
    except Exception as e:
        print(f"Error saving image: {e}")


In [26]:
save_img_to_file(test_images[12])

Image saved to input


In [27]:
# Example usage
digits, error = load_img_from_file(show=True)

----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
--------------&&&-----------
-------------&&&&&--&-------
-----------&&------&&-------
----------&&-------&--------
----------&-------&&--------
---------&&-------&---------
---------&-------&&---------
---------&------&&----------
---------&&----&&&----------
----------&&&&&--&----------
----------------&-----------
---------------&&-----------
---------------&&-----------
---------------&------------
--------------&&------------
--------------&&------------
--------------&-------------
--------------&-------------
--------------&-------------
----------------------------
----------------------------



## Test

In [48]:
import psutil
import time
import os
import subprocess, re

# Function to get current memory usage
def get_memory_usage():
    return psutil.Process(os.getpid()).memory_info().rss / (1024 * 1024)  # Memory in MB

In [30]:
vm_file = "../bin/vm"
program = "../bin/gglm.bin"
model = "../models/ggml-model-f32-big-endian.bin"
#input_img = data_file
folder = "./tmp/"

# Create the directory 'tmp' in the current working directory
try:
    os.makedirs(folder, exist_ok=True)
    print(f"Directory '{folder}' created successfully")
except OSError as error:
    print(f"Directory '{folder}' cannot be created. Error: {error}")

Directory './tmp/' created successfully


In [55]:
subprocess.run([f"{vm_file}", f"--basedir={folder}",
                f"--program={program}", f"--model={model}", f"--data={data_file}", "--mipsVMCompatible"])

# Calculate the end memory and time
end_memory = get_memory_usage()
end_time = time.time()

# Calculate and print the usage
print(f"Memory used: {end_memory - start_memory} MB")
print(f"Execution time: {end_time - start_time} seconds")

modelSize:  178164
rawSize:  [0 2 183 244]


Start MIPS MNIST
start MIPS_InputProcess
buf len:  784
____________________________
____________________________
____________________________
____________________________
_______________***__________
____________****____________
________***_________________
_________*__________________
__________*_________________
__________*_________________
__________*_________________
__________*****_____________
_________*______**__________
________*_________*_________
____________________*_______
_____________________*______
______________________*_____
_______________________*____
_______________________*____
______________________*_____
______________________*_____
________*___________*_______
________***____****_________
___________**_______________
____________________________
____________________________
____________________________
____________________________

start MIPS_mnist_model_load
model_bytes len:  178164
magic: 67676d6c
reading fc1
n_dims:  2
ne_weight:  [784 56]
index:  175632
read

reach the final state, total step: 32835652, target: -1
writing ./tmp//checkpoint_32835652.json len 7583300 with root 0x4bd7ade2636d4db80eea591c1afe437f290b38a952a7a159c8bb2a3ec808146f
lastStep:  32835652
writing ./tmp//checkpoint_final.json len 7583300 with root 0x4bd7ade2636d4db80eea591c1afe437f290b38a952a7a159c8bb2a3ec808146f
PC: 5ead0000
Memory used: 0.0 MB
Execution time: 31.350303411483765 seconds


In [35]:
save_img_to_file(test_images[15])
_ = load_img_from_file(show=True)

Image saved to input
----------------------------
----------------------------
----------------------------
----------------------------
---------------&&&----------
------------&&&&------------
--------&&&-----------------
---------&------------------
----------&-----------------
----------&-----------------
----------&-----------------
----------&&&&&-------------
---------&------&&----------
--------&---------&---------
--------------------&-------
---------------------&------
----------------------&-----
-----------------------&----
-----------------------&----
----------------------&-----
----------------------&-----
--------&-----------&-------
--------&&&----&&&&---------
-----------&&---------------
----------------------------
----------------------------
----------------------------
----------------------------



In [54]:
# Start memory and time monitoring
start_memory = get_memory_usage()
start_time = time.time()

command = [f"{vm_file}", "--lastLayer", f"--basedir={folder}",
                f"--program={program}", f"--model={model}", f"--data={data_file}", "--mipsVMCompatible"]


# Start the subprocess
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Wait for the subprocess to finish and capture the output
stdout, stderr = process.communicate()

print (stdout)

# Decode the output to a string
output = stdout.decode()

# Calculate the end memory and time
end_memory = get_memory_usage()
end_time = time.time()


# Calculate and print the usage
print(f"Memory used: {end_memory - start_memory} MB")
print(f"Execution time: {end_time - start_time} seconds")

print (output)

# Find the line with the predicted digit using a regular expression
match = re.search(r"Predicted digit is", output)
if match:
    predicted_digit = int(match.group(1))
    print(f"Predicted digit is {predicted_digit}")
else:
    print("Predicted digit not found in the output")

b'modelSize:  178164\nrawSize:  [0 2 183 244]\nreach the final state, total step: 32835652, target: -1\nwriting ./tmp//checkpoint_32835652.json len 7583300 with root 0x4bd7ade2636d4db80eea591c1afe437f290b38a952a7a159c8bb2a3ec808146f\nlastStep:  32835652\nwriting ./tmp//checkpoint_final.json len 7583300 with root 0x4bd7ade2636d4db80eea591c1afe437f290b38a952a7a159c8bb2a3ec808146f\nPC: 5ead0000\n'
Memory used: 0.0 MB
Execution time: 3.4775969982147217 seconds
modelSize:  178164
rawSize:  [0 2 183 244]
reach the final state, total step: 32835652, target: -1
writing ./tmp//checkpoint_32835652.json len 7583300 with root 0x4bd7ade2636d4db80eea591c1afe437f290b38a952a7a159c8bb2a3ec808146f
lastStep:  32835652
writing ./tmp//checkpoint_final.json len 7583300 with root 0x4bd7ade2636d4db80eea591c1afe437f290b38a952a7a159c8bb2a3ec808146f
PC: 5ead0000

Predicted digit not found in the output
